In [1]:
from keras.applications import ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model


94765736/94765736 [==============================] - 2s 0us/step


In [2]:
from tensorflow.keras import layers, models

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Use 1 output unit for binary classification (fall/no-fall)
])


In [3]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [8]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_and_copy_files(files, dest_train, dest_val, dest_test, val_size=0.1, test_size=0.2):
    """
    Split files into train, validation, and test sets and move them to respective directories.
    
    Args:
        files: List of file paths.
        dest_train: Directory to save training files.
        dest_val: Directory to save validation files.
        dest_test: Directory to save test files.
        val_size: Fraction of files for validation.
        test_size: Fraction of files for test.
    """
    # Split the data into training, test, and validation
    train_files, test_files = train_test_split(files, test_size=test_size, random_state=42)
    train_files, val_files = train_test_split(train_files, test_size=val_size, random_state=42)

    # Move the files to their respective directories
    for file in train_files:
        shutil.copy(file, dest_train)
    
    for file in val_files:
        shutil.copy(file, dest_val)

    for file in test_files:
        shutil.copy(file, dest_test)

    print(f"Moved {len(train_files)} files to train, {len(val_files)} files to validation, and {len(test_files)} files to test.")


def organize_and_split_frames(video_frames_path, annotations_path, base_save_dir, val_size=0.1, test_size=0.2):
    """
    Organize frames by fall status and split them into train, validation, and test sets.
    
    Args:
        video_frames_path: Directory where video frames are stored.
        annotations_path: Path to the annotation files.
        base_save_dir: Base directory where train, validation, and test directories will be created.
        val_size: Fraction of data to be used for validation.
        test_size: Fraction of data to be used for test.
    """
    # Directories to save organized frames
    train_fall_dir = os.path.join(base_save_dir, 'train', 'fall')
    val_fall_dir = os.path.join(base_save_dir, 'validation', 'fall')
    test_fall_dir = os.path.join(base_save_dir, 'test', 'fall')
    
    train_no_fall_dir = os.path.join(base_save_dir, 'train', 'no_fall')
    val_no_fall_dir = os.path.join(base_save_dir, 'validation', 'no_fall')
    test_no_fall_dir = os.path.join(base_save_dir, 'test', 'no_fall')
    
    # Create directories if they don't exist
    os.makedirs(train_fall_dir, exist_ok=True)
    os.makedirs(val_fall_dir, exist_ok=True)
    os.makedirs(test_fall_dir, exist_ok=True)
    
    os.makedirs(train_no_fall_dir, exist_ok=True)
    os.makedirs(val_no_fall_dir, exist_ok=True)
    os.makedirs(test_no_fall_dir, exist_ok=True)

    fall_files = []
    no_fall_files = []
    
    for annotation_file in os.listdir(annotations_path):
        video_name = os.path.splitext(annotation_file)[0]
        frame_dir = os.path.join(video_frames_path, video_name)
        
        if not os.path.exists(frame_dir):
            print(f"Frames directory for {video_name} not found.")
            continue
        
        annotation_file_path = os.path.join(annotations_path, annotation_file)
        with open(annotation_file_path, "r") as f:
            for line in f:
                parts = line.strip().split(",")
                if len(parts) < 2:
                    continue
                
                frame_number = parts[0]
                fall_indicator = int(parts[1])  # Assuming 1 means fall, 0 means no fall
                
                frame_filename = f"{frame_number}.jpg"
                frame_path = os.path.join(frame_dir, frame_filename)
                
                if not os.path.exists(frame_path):
                    print(f"Frame {frame_filename} not found in {frame_dir}.")
                    continue
                
                # Categorize frames into fall or no_fall
                if fall_indicator == 1:
                    fall_files.append(frame_path)
                else:
                    no_fall_files.append(frame_path)
    
    # Split and copy fall frames
    split_and_copy_files(fall_files, train_fall_dir, val_fall_dir, test_fall_dir, val_size, test_size)

    # Split and copy no_fall frames
    split_and_copy_files(no_fall_files, train_no_fall_dir, val_no_fall_dir, test_no_fall_dir, val_size, test_size)


# Example usage:
video_frames_directory = "datasets/FDD/Coffee_room_01/Videos_with_frames"
annotations_directory = "datasets/FDD/Coffee_room_01/Coffee_room_01/Annotation_files"
base_directory = "fall_detection_data"  # Base directory where train, validation, test folders will be created

organize_and_split_frames(video_frames_directory, annotations_directory, base_directory)


Frame 1.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (1).
Frame 2.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (1).
Frame 3.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (1).
Frame 4.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (1).
Frame 1.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (3).
Frame 2.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (3).
Frame 3.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (3).
Frame 4.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (3).
Frame 5.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (3).
Frame 1.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (4).
Frame 2.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (4).
Frame 3.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (4).
Fram

In [9]:
import os
import shutil

def organize_frames_by_fall_status(video_frames_path, annotations_path, fall_save_dir, no_fall_save_dir):
    """
    Organize frames into fall and no_fall directories based on the annotations.
    
    Args:
        video_frames_path: Directory where video frames are stored (subdirectories for each video).
        annotations_path: Path to the annotation files.
        fall_save_dir: Directory to save frames with falls.
        no_fall_save_dir: Directory to save frames without falls.
    """
    os.makedirs(fall_save_dir, exist_ok=True)
    os.makedirs(no_fall_save_dir, exist_ok=True)
    
    for annotation_file in os.listdir(annotations_path):
        video_name = os.path.splitext(annotation_file)[0]  # Extract video name without extension
        
        frame_dir = os.path.join(video_frames_path, video_name)  # e.g., video_frames_path/video1
        
        if not os.path.exists(frame_dir):
            print(f"Frames directory for {video_name} not found.")
            continue
        
        annotation_file_path = os.path.join(annotations_path, annotation_file)
        with open(annotation_file_path, "r") as f:
            for line in f:
                parts = line.strip().split(",")
                if len(parts) < 2:
                    continue
                
                frame_number = parts[0]  # The frame number from annotation
                fall_indicator = int(parts[1])  # Assuming 1 means fall, 0 means no fall
                
                frame_filename = f"{frame_number}.jpg"  # Expecting frame numbers match this format
                frame_path = os.path.join(frame_dir, frame_filename)  # Construct full path to frame
                
                if not os.path.exists(frame_path):
                    print(f"Frame {frame_filename} not found in {frame_dir}.")
                    continue
                
                # Move frame to the appropriate directory based on fall or no_fall
                if fall_indicator == 1:
                    shutil.move(frame_path, os.path.join(fall_save_dir, frame_filename))
                else:
                    shutil.move(frame_path, os.path.join(no_fall_save_dir, frame_filename))
                
                print(f"Moved {frame_filename} to {'fall' if fall_indicator == 1 else 'no_fall'} category.")

# Example usage:
video_frames_directory = "datasets/FDD/Coffee_room_01/Videos_with_frames"  # Directory containing video folders
annotations_directory = "datasets/FDD/Coffee_room_01/Coffee_room_01/Annotation_files"  # Annotations path
fall_directory = "fall_detection_data/train/fall"
no_fall_directory = "fall_detection_data/train/no_fall"

organize_frames_by_fall_status(video_frames_directory, annotations_directory, fall_directory, no_fall_directory)


Frame 1.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (1).
Frame 2.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (1).
Frame 3.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (1).
Frame 4.jpg not found in datasets/FDD/Coffee_room_01/Videos_with_frames\video (1).
Moved 5.jpg to fall category.
Moved 6.jpg to fall category.
Moved 7.jpg to fall category.
Moved 8.jpg to fall category.
Moved 9.jpg to fall category.
Moved 10.jpg to fall category.
Moved 11.jpg to fall category.
Moved 12.jpg to fall category.
Moved 13.jpg to fall category.
Moved 14.jpg to fall category.
Moved 15.jpg to fall category.
Moved 16.jpg to fall category.
Moved 17.jpg to fall category.
Moved 18.jpg to fall category.
Moved 19.jpg to fall category.
Moved 20.jpg to fall category.
Moved 21.jpg to fall category.
Moved 22.jpg to fall category.
Moved 23.jpg to fall category.
Moved 24.jpg to fall category.
Moved 25.jpg to fall category.
Moved 26.jpg to fall c

testing